In [89]:
## Source for Cases Data: https://github.com/nytimes/covid-19-data.git

## Source for mobility Data: https://pastelsky.github.io/covid-19-mobility-tracker

In [90]:
# The code below gets the covid-19 cases/deaths data at both country ans state level for USA.

# It also extracts the US Mobility data at the country level and state level.

# The data is then merged to analyze the impact of mobility/social distancing on the covid-19 case count.

In [91]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import requests

import json

from us_state_abbrev import us_state_abbrev

In [92]:
# Input File paths

file_us = '../covid-19-data/us.csv'

file_states = '../covid-19-data/us-states.csv'

file_counties = '../covid-19-data/us-counties.csv'

In [93]:
# Create Pandas Data Frame

us_covid_df = pd.read_csv(file_us)

us_states_df = pd.read_csv(file_states)

In [94]:
# Country Level

us_covid_df.head()

,date,cases,deaths
0,2020-01-21,1,0
1,2020-01-22,1,0
2,2020-01-23,1,0
3,2020-01-24,2,0
4,2020-01-25,3,0


In [95]:
duplicates = us_states_df['state'].tolist()

state_list = list(set(duplicates))

state_list.sort()

In [96]:
# Create a dictionary to store all the observations for each state

df = [state for _, state in us_states_df.groupby('state')]


data_frame = dict(zip(state_list, df)) 

In [97]:
# Sample to extract data for state

data_frame['Georgia'].head()

,date,state,fips,cases,deaths
256,2020-03-02,Georgia,13,2,0
271,2020-03-03,Georgia,13,2,0
287,2020-03-04,Georgia,13,2,0
305,2020-03-05,Georgia,13,2,0
326,2020-03-06,Georgia,13,3,0


In [98]:
# URL TO get mobility data for the entire country

url = "https://pastelsky.github.io/covid-19-mobility-tracker/output/US/mobility.json"

response = requests.get(url).json()

In [99]:
# Extract Mobility data for retail & recreation

locations = ['retailAndRecreation', 'parks', 'groceryAndPharmacy', 'transitStations','workplaces', 'residential']

location_df = []

for location in locations:
    
    request = response['country'][location]['points']
    
    location_df.append(request)

# retail & recreation

retail = location_df[0]

retail_df = pd.DataFrame(retail)

retail_df.rename(columns = {'value':'Per_decrease_ret_rec'}, inplace = True)


# Parks

parks = location_df[1]

parks_df = pd.DataFrame(parks)

parks_df.rename(columns = {'value':'Per_decrease_parks'}, inplace = True)


# grocery

grocery = location_df[2]

grocery_df = pd.DataFrame(grocery)

grocery_df.rename(columns = {'value':'Per_decrease_grocery'}, inplace = True)

# transit

transit = location_df[3]

transit_df = pd.DataFrame(transit)

transit_df.rename(columns = {'value':'Per_decrease_transit'}, inplace = True)


# Work place

workplace = location_df[4]

workplace_df = pd.DataFrame(workplace)

workplace_df.rename(columns = {'value':'Per_decrease_workplace'}, inplace = True)

# Residential

residential = location_df[5]

residential_df = pd.DataFrame(residential)

residential_df.rename(columns = {'value':'Per_decrease_residential'}, inplace = True)

In [100]:
# Create the final Mobility data
mobility_df = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(retail_df, parks_df, on = 'date', how = 'outer'),
        grocery_df, on = 'date', how = 'outer'),transit_df, on = 'date', how = 'outer'),
        workplace_df, on = 'date', how = 'outer'),residential_df, on = 'date', how = 'outer')

mobility_df.sort_values('date').head()

,date,Per_decrease_ret_rec,Per_decrease_parks,Per_decrease_grocery,Per_decrease_transit,Per_decrease_workplace,Per_decrease_residential
49,2020-02-16,6,28,0,-9,-23,5
48,2020-02-17,1,7,0,1,-2,1
47,2020-02-18,2,8,0,1,1,0
46,2020-02-19,1,5,0,0,0,1
45,2020-02-20,2,4,-2,1,0,0


In [101]:
# Merge cases data with mobility

combined_df = pd.merge(mobility_df, us_covid_df, on = 'date', how = 'left')

combined_df = combined_df.sort_values('date')

combined_df = combined_df[['date','cases', 'deaths', 'Per_decrease_ret_rec', 'Per_decrease_parks', 'Per_decrease_grocery',
                           'Per_decrease_transit', 'Per_decrease_workplace', 'Per_decrease_residential']]

In [102]:
combined_df.head()

,date,cases,deaths,Per_decrease_ret_rec,Per_decrease_parks,Per_decrease_grocery,Per_decrease_transit,Per_decrease_workplace,Per_decrease_residential
49,2020-02-16,15,0,6,28,0,-9,-23,5
48,2020-02-17,25,0,1,7,0,1,-2,1
47,2020-02-18,25,0,2,8,0,1,1,0
46,2020-02-19,25,0,1,5,0,0,0,1
45,2020-02-20,27,0,2,4,-2,1,0,0


In [103]:
# Create a data frame with State and it's abbrevation

us_state_abbrev_df = pd.DataFrame(us_state_abbrev.items(), columns=['state', 'Abrevation'])

us_state_abbrev_list = us_state_abbrev.values()

In [104]:
us_state_abbrev_list

df = pd.DataFrame()

for state in us_state_abbrev_list:
    
    mobility_states_df = pd.DataFrame()
    
    try:
        
        url_t = (f"https://pastelsky.github.io/covid-19-mobility-tracker/output/US/{state}/mobility.json")
        
        results_t = requests.get(url_t).json()
        
        print(f"Mobility data found for {state}")
        
        # Retail
        
        retail_list = results_t['state']['retailAndRecreation']['points']
        
        date_retail = [date['date'] for date in retail_list ]
        
        mobility_states_df['state']  = [(f"{state}") for date in retail_list]
        
        value_retail = [value['value'] for value in retail_list ]
        
        mobility_states_df['date'] = date_retail
        
        mobility_states_df['Per_decrease_ret_rec'] = value_retail
        
        # Parks
        
        park_list = results_t['state']['parks']['points']
        
        value_park = [value['value'] for value in park_list ]
        
        mobility_states_df['Per_decrease_parks'] = value_park
        
        # Grocery
        
        grocery_list = results_t['state']['groceryAndPharmacy']['points']
        
        value_grocery = [value['value'] for value in grocery_list ]
        
        mobility_states_df['Per_decrease_grocery'] = value_grocery
        
        #Transit
        
        transit_list = results_t['state']['transitStations']['points']
        
        value_transit = [value['value'] for value in transit_list ]
        
        mobility_states_df['Per_decrease_transit'] = value_transit
        
        # Workplaces
        
        work_list = results_t['state']['workplaces']['points']
        
        value_work = [value['value'] for value in work_list ]
        
        mobility_states_df['Per_decrease_workplace'] = value_work
        
        
        #Residential
        
        resi_list = results_t['state']['residential']['points']
        
        value_resi = [value['value'] for value in resi_list ]
        
        mobility_states_df['Per_decrease_residential'] = value_resi 
        
        df = df.append(mobility_states_df)
        
    except:
        
        print(f"Mobility data not found for {state}")
        

Mobility data found for AL
Mobility data found for AK
Mobility data not found for AS
Mobility data found for AZ
Mobility data found for AR
Mobility data found for CA
Mobility data found for CO
Mobility data found for CT
Mobility data found for DE
Mobility data not found for DC
Mobility data found for FL
Mobility data found for GA
Mobility data not found for GU
Mobility data found for HI
Mobility data found for ID
Mobility data found for IL
Mobility data found for IN
Mobility data found for IA
Mobility data found for KS
Mobility data found for KY
Mobility data found for LA
Mobility data found for ME
Mobility data found for MD
Mobility data found for MA
Mobility data found for MI
Mobility data found for MN
Mobility data found for MS
Mobility data found for MO
Mobility data found for MT
Mobility data found for NE
Mobility data found for NV
Mobility data found for NH
Mobility data found for NJ
Mobility data found for NM
Mobility data found for NY
Mobility data found for NC
Mobility data fo

In [105]:
# Reseting the index

mobility_states_final = df.reset_index(drop = True)

In [106]:
# Getting the state abbrevations to merge with mobility data

us_states_df2 = pd.merge(us_states_df, us_state_abbrev_df, on = 'state', how = 'left')

us_states_df2 = us_states_df2[['date', 'Abrevation', 'fips', 'cases', 'deaths']]

us_states_df2 = us_states_df2.rename(columns = {'Abrevation': 'state'})

In [107]:
us_states_df2.head()

,date,state,fips,cases,deaths
0,2020-01-21,WA,53,1,0
1,2020-01-22,WA,53,1,0
2,2020-01-23,WA,53,1,0
3,2020-01-24,IL,17,1,0
4,2020-01-24,WA,53,1,0


In [108]:
# Merge Cased data with mobility data

combined_states_df = pd.merge(mobility_states_final, us_states_df2, how = 'left', 
                              left_on = ['date', 'state'], right_on = ['date', 'state'])

combined_states_df = combined_states_df[['state','date','cases', 'deaths', 'Per_decrease_ret_rec', 'Per_decrease_parks', 'Per_decrease_grocery',
                           'Per_decrease_transit', 'Per_decrease_workplace', 'Per_decrease_residential']]

combined_states_df = combined_states_df.sort_values(['state', 'date'], ascending=[True, True])



In [109]:
# Export the data Excel file

combined_df.to_excel('../data/combined_df_US.xlsx', index = False)

combined_states_df.to_excel('../data/combined_df_states.xlsx', index = False)